### Setup

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import sys

if 'google.colab' in sys.modules:
    %pip install -q -U keras_tuner

In [ ]:
%pip install 'git+https://github.com/jeslago/epftoolbox.git'

  Cloning https://github.com/jeslago/epftoolbox.git to /tmp/pip-req-build-8bc2kx7x
  Running command git clone --filter=blob:none --quiet https://github.com/jeslago/epftoolbox.git /tmp/pip-req-build-8bc2kx7x
  Resolved https://github.com/jeslago/epftoolbox.git to commit 47d6e0629f65ebd19d3c12cb5689dbad0c2ea078
  Preparing metadata (setup.py) ... done
  Created wheel for epftoolbox: filename=epftoolbox-1.0-py3-none-any.whl size=58746 sha256=2a70f88c45fb712e0cb2b862492e3a0ea00ee0f3811e32994822f01a71c5828d
  Stored in directory: /tmp/pip-ephem-wheel-cache-lgh_9hi8/wheels/f1/d7/c7/feb15c8f047f5b069656a52f3bcde89a7310a311f9cfac292a
Successfully built epftoolbox


In [ ]:
%pip install finta

In [ ]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.3 MB/s eta 0:00:00


In [ ]:
%pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import sys
from pathlib import Path

import finta
import keras_tuner as kt
import optuna
import optuna_integration
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import sklearn
import statsmodels
import tensorflow as tf
import tqdm
from google.colab import drive
from sklearn.linear_model import LinearRegression

In [ ]:
drive.mount('/content/drive')

DRIVE_ROOT = Path('/content/drive/MyDrive')

PROJECT_ROOT = DRIVE_ROOT / 'projects/btc'
PROJECT_DATA_DIR = PROJECT_ROOT / 'data'
PROJECT_CLEANED_DIR = PROJECT_DATA_DIR / 'cleaned'
PROJECT_EXTERNAL_DIR = PROJECT_DATA_DIR / 'external'

CONFIG_FILENAME = 'config.yaml'
BTC_CLEANED_CSV_FILENAME = 'ohlcv_274_cleaned.csv'
FEDFUNDRATE_CSV_FILENAME = 'fedfunds.csv'
M2SL_CSV_FILENAME = 'm2sl.csv'

CONFIG_FILE_PATH = PROJECT_ROOT / CONFIG_FILENAME
BTC_CLEANED_FILE_PATH = PROJECT_CLEANED_DIR / BTC_CLEANED_CSV_FILENAME
FEDFUNDRATE_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / FEDFUNDRATE_CSV_FILENAME
M2SL_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / M2SL_CSV_FILENAME

Mounted at /content/drive


In [ ]:
project_root = os.path.abspath(PROJECT_ROOT)

if project_root not in sys.path:
  sys.path.append(project_root)

print(f'Project root \'{project_root}\' added to sys.path.')

Project root '/content/drive/MyDrive/projects/btc' added to sys.path.


In [ ]:
from src.analysis import *
from src.data_loader import *
from src.data_splitter import *
from src.data_cleaner import *
from src.model import *
from src.preprocessing import *
from src.utils import *
from src.visualization import *

In [ ]:
print('--- Python Interpreter and Standard Library Versions ---')
print(f'- python: {sys.version}')
print(f'- python info: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

print(f'- os, hashlib, pathlib module: (version tied to Python interpreter)')

print('\n--- Third-Party Library Versions ---')
print(f'- keras_tuner: {kt.__version__}')
print(f'- optuna: {optuna.__version__}')
print(f'- matplotlib: {matplotlib.__version__}')
print(f'- numpy: {np.__version__}')
print(f'- pandas: {pd.__version__}')
print(f'- seaborn: {seaborn.__version__}')
print(f'- sklearn: {sklearn.__version__}')
print(f'- statsmodels: {statsmodels.__version__}')
print(f'- tensorflow: {tf.__version__}')
#print(f'- tqdm: {tqdm.__version__}')

print('\n--- Google Colab Specific Module Versions ---')
print(f'- google.colab.drive: (version tied to Colab environment)')

--- Python Interpreter and Standard Library Versions ---
- python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
- python info: 3.12.12
- os, hashlib, pathlib module: (version tied to Python interpreter)

--- Third-Party Library Versions ---
- keras_tuner: 1.4.7
- optuna: 4.5.0
- matplotlib: 3.10.0
- numpy: 2.0.2
- pandas: 2.2.2
- seaborn: 0.13.2
- sklearn: 1.6.1
- statsmodels: 0.14.5
- tensorflow: 2.19.0

--- Google Colab Specific Module Versions ---
- google.colab.drive: (version tied to Colab environment)


In [ ]:
%pip show epftoolbox

Name: epftoolbox
Version: 1.0
Summary: An open-access benchmark and toolbox for electricity price forecasting
Home-page: https://github.com/jeslago/epftoolbox
Author: Jesus Lago
Author-email: jesuslagogarcia@gmail.com
License: GNU AGPLv3
Location: /usr/local/lib/python3.12/dist-packages
Requires: hyperopt, keras, matplotlib, numpy, pandas, scikit-learn, scipy, statsmodels, tensorflow
Required-by: 


In [ ]:
%pip show finta

Name: finta
Version: 1.3
Summary: Common financial technical indicators implemented in Pandas.
Home-page: https://github.com/peerchemist/finta
Author: Peerchemist
Author-email: peerchemist@protonmail.ch
License: LGPLv3+
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, pandas
Required-by: 


### Load the cleaned dataset.

In [ ]:
cleaned_ds = load_btc_ds(BTC_CLEANED_FILE_PATH, 'date', True)

FileNotFoundError: Error: The file was not found at /content/drive/MyDrive/projects/btc/data/cleaned/ohlcv_274_cleaned.csv

### Perform linear interpolation to fill the missing values on March 15, 2025, to ensure a fair and correct comparison.

In [ ]:
imputed_ds = fill_time_series_gaps(cleaned_ds, 'min')

### Resample the cleaned dataset into multiple timeframes.

In [ ]:
resampled_ds = resample_btc_data(imputed_ds)

### Split the resampled datasets into training, validation, and test sets of the target timeframe.

In [ ]:
train_ds, valid_ds, test_ds = split_btc_ds(resampled_ds['daily'], 'Day')

### Prepare the dataset by consolidating data splits, adding a comprehensive suite of features, and then splitting the data back.

In [ ]:
train_prep_ds, valid_prep_ds, test_prep_ds = prepare_feature_ds(
    train_ds, valid_ds, test_ds,
    ['open', 'high', 'low', 'close', 'volume'],
    FEDFUNDRATE_CSV_FILE_PATH, M2SL_CSV_FILE_PATH
)

### Model Comparisons (1-Week)

In [ ]:
tf.random.set_seed(42)

In [ ]:
FULL_COLS = list(train_prep_ds.columns)
TARGET_COL = 'close'
WINDOW_SIZE = 7
INPUT_WINDOW = 7
TARGET_WINDOW = 7
FORECAST_HORIZON = 7

In [ ]:
model_hps = load_config(CONFIG_FILE_PATH)

Configuration successfully loaded from '/content/drive/MyDrive/projects/btc/config.yaml'.


In [ ]:
naive_metrics, naive_pred_out = train_and_forecast_with_naive_model(test_prep_ds, TARGET_COL, FORECAST_HORIZON, 'Test', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Test Set (Horizon: 7 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 5.0442%
- Directional Accuracy (DA): 47.1816%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2025-06-29: $102958.00
- Directional Signal for 2025-06-29: 1.0


In [ ]:
plot_residuals_analysis(naive_metrics, 'comp', 14, 'NAIVE', 'Week')
run_ljung_box_test(naive_metrics, 'comp', 14, 'NAIVE')
print(naive_metrics['comp'])

Plot saved to 'outputs/plots/naive_residuals_analysis_week'.
= Ljung-Box Test: NAIVE =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
                true      pred
2024-02-23   51094.0   51993.0
2024-02-24   51429.0   51278.0
2024-02-25   51566.0   51805.0
2024-02-26   53644.0   51934.0
2024-02-27   57165.0   51506.0
...              ...       ...
2025-06-18  104922.0  108678.0
2025-06-19  104688.0  105731.0
2025-06-20  103316.0  106118.0
2025-06-21  102139.0  105464.0
2025-06-22  102958.0  105617.0

[486 rows x 2 columns]


In [ ]:
arima_order = tuple(model_hps['ARIMA']['weekly']['order'])
arima_metrics, arima_pred_out = train_and_forecast_with_best_arima_model(train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                         arima_order, TARGET_COL, WINDOW_SIZE,
                                                                         FORECAST_HORIZON, 'Days')

= ARIMA Model Training, Evaluation, and Forecasting on Test Set (Horizon: 7 Days) =
--- Fitting ARIMA(5, 1, 3) Model ---

--- Evaluating ARIMA(5, 1, 3) Model ---

- Order: (5, 1, 3)
- Mean Absolute Percentage Error (MAPE): 5.3620%
- Directional Accuracy (DA): 52.8807%

--- Generating Final ARIMA(5, 1, 3) (Threshold: 0.5%) Forecast ---
- Forecast for 2025-06-29: $102860.92
- Directional Signal for 2025-06-29: 0.0


In [ ]:
plot_residuals_analysis(arima_metrics, 'comp', 14, 'ARIMA', 'Week')
run_ljung_box_test(arima_metrics, 'comp', 14, 'ARIMA')
print(arima_metrics['comp'])

Plot saved to 'outputs/plots/arima_residuals_analysis_week'.
= Ljung-Box Test: ARIMA =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
                true           pred
2024-02-23   51094.0   52306.892992
2024-02-24   51429.0   52268.656187
2024-02-25   51566.0   51521.079588
2024-02-26   53644.0   52235.285836
2024-02-27   57165.0   51918.295400
...              ...            ...
2025-06-18  104922.0  110882.379948
2025-06-19  104688.0  109108.343416
2025-06-20  103316.0  106252.947514
2025-06-21  102139.0  106625.468926
2025-06-22  102958.0  104942.255097

[486 rows x 2 columns]


In [ ]:
lstm_hps = kt.HyperParameters()
lstm_hps.values = model_hps['LSTM']['weekly']

lstm_metrics, lstm_pred_out = train_and_forecast_with_best_rnn_model('lstm', train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                    lstm_hps, FULL_COLS, TARGET_COL, FORECAST_HORIZON,
                                                                    INPUT_WINDOW, TARGET_WINDOW, 'Days')

= LSTM Model Training, Evaluation, and Forecasting on Test Set (Horizon: 7 Days) =
--- Preparing Data for LSTM ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, None, 96)       │         9,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 192)      │        92,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 96)       │       110,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, None, 96)       │           192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 160)      │       164,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, None, 160)      │           320 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, None, 32)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, None, 32)       │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 7)        │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 403,015 (1.54 MB)

 Trainable params: 403,015 (1.54 MB)

 Non-trainable params: 0 (0.00 B)


--- Fitting LSTM Model ---
Epoch 1/100
136/136 - 26s - 190ms/step - loss: 0.3816 - mae: 0.7034 - val_loss: 0.1749 - val_mae: 0.4507
Epoch 2/100
136/136 - 9s - 67ms/step - loss: 0.3199 - mae: 0.6125 - val_loss: 0.1744 - val_mae: 0.4490
Epoch 3/100
136/136 - 11s - 77ms/step - loss: 0.3191 - mae: 0.6108 - val_loss: 0.1751 - val_mae: 0.4512
Epoch 4/100
136/136 - 10s - 77ms/step - loss: 0.3192 - mae: 0.6111 - val_loss: 0.1745 - val_mae: 0.4492
Epoch 5/100
136/136 - 11s - 80ms/step - loss: 0.3193 - mae: 0.6112 - val_loss: 0.1739 - val_mae: 0.4473
Epoch 6/100
136/136 - 10s - 70ms/step - loss: 0.3196 - mae: 0.6116 - val_loss: 0.1743 - val_mae: 0.4466
Epoch 7/100
136/136 - 10s - 74ms/step - loss: 0.3196 - mae: 0.6116 - val_loss: 0.1738 - val_mae: 0.4463
Epoch 8/100
136/136 - 10s - 77ms/step - loss: 0.3184 - mae: 0.6102 - val_loss: 0.1741 - val_mae: 0.4465
Epoch 9/100
136/136 - 20s - 150ms/step - loss: 0.3180 - mae: 0.6091 - val_loss: 0.1739 - val_mae: 0.4465
Epoch 10/100
136/136 - 9s - 67ms/st

Walk-Forward Validation: 100%|██████████| 486/486 [00:38<00:00, 12.69it/s]


- Mean Absolute Percentage Error (MAPE): 5.0402%
- Directional Accuracy (DA): 50.8230%

--- Generating Final LSTM (Threshold: 0.5%) Forecast ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
- Forecast for 2025-06-29: $103112.36
- Directional Signal for 2025-06-29: 0.0


In [ ]:
plot_residuals_analysis(lstm_metrics, 'comp', 14, 'LSTM', 'Week')
run_ljung_box_test(lstm_metrics, 'comp', 14, 'LSTM')
print(lstm_metrics['comp'])

Plot saved to 'outputs/plots/lstm_residuals_analysis_week'.
= Ljung-Box Test: LSTM =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
                true           pred
2024-02-23   51094.0   52070.948636
2024-02-24   51429.0   51354.876706
2024-02-25   51566.0   51882.666787
2024-02-26   53644.0   52011.860172
2024-02-27   57165.0   51583.218522
...              ...            ...
2025-06-18  104922.0  108840.931583
2025-06-19  104688.0  105889.513400
2025-06-20  103316.0  106277.093604
2025-06-21  102139.0  105622.113059
2025-06-22  102958.0  105775.342489

[486 rows x 2 columns]


In [ ]:
gru_hps = kt.HyperParameters()
gru_hps.values = model_hps['GRU']['weekly']

gru_metrics, gru_pred_out = train_and_forecast_with_best_rnn_model('gru', train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                   gru_hps, FULL_COLS, TARGET_COL, FORECAST_HORIZON,
                                                                   INPUT_WINDOW, TARGET_WINDOW, 'Days')

= GRU Model Training, Evaluation, and Forecasting on Test Set (Horizon: 7 Days) =
--- Preparing Data for GRU ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, None, 96)       │         9,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, None, 192)      │        92,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, None, 96)       │        83,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, None, 96)       │           192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, None, 160)      │       123,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (None, None, 160)      │           320 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, None, 32)       │        18,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_5           │ (None, None, 32)       │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, None, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 7)        │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 328,839 (1.25 MB)

 Trainable params: 328,839 (1.25 MB)

 Non-trainable params: 0 (0.00 B)


--- Fitting GRU Model ---
Epoch 1/100
136/136 - 22s - 160ms/step - loss: 1.0352 - mae: 0.7274 - val_loss: 0.1862 - val_mae: 0.4477
Epoch 2/100
136/136 - 9s - 63ms/step - loss: 0.3267 - mae: 0.6156 - val_loss: 0.1774 - val_mae: 0.4496
Epoch 3/100
136/136 - 10s - 70ms/step - loss: 0.3222 - mae: 0.6117 - val_loss: 0.1759 - val_mae: 0.4492
Epoch 4/100
136/136 - 10s - 72ms/step - loss: 0.3214 - mae: 0.6118 - val_loss: 0.1752 - val_mae: 0.4482
Epoch 5/100
136/136 - 10s - 76ms/step - loss: 0.3203 - mae: 0.6108 - val_loss: 0.1751 - val_mae: 0.4480
Epoch 6/100
136/136 - 9s - 63ms/step - loss: 0.3202 - mae: 0.6106 - val_loss: 0.1745 - val_mae: 0.4468
Epoch 7/100
136/136 - 10s - 71ms/step - loss: 0.3197 - mae: 0.6107 - val_loss: 0.1747 - val_mae: 0.4474
Epoch 8/100
136/136 - 10s - 70ms/step - loss: 0.3201 - mae: 0.6107 - val_loss: 0.1746 - val_mae: 0.4468
Epoch 9/100
136/136 - 8s - 62ms/step - loss: 0.3193 - mae: 0.6091 - val_loss: 0.1742 - val_mae: 0.4465
Epoch 10/100
136/136 - 10s - 71ms/step 

Walk-Forward Validation: 100%|██████████| 486/486 [00:39<00:00, 12.27it/s]

- Mean Absolute Percentage Error (MAPE): 5.0401%
- Directional Accuracy (DA): 50.8230%

--- Generating Final GRU (Threshold: 0.5%) Forecast ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
- Forecast for 2025-06-29: $103174.38
- Directional Signal for 2025-06-29: 0.0


In [ ]:
plot_residuals_analysis(gru_metrics, 'comp', 14, 'GRU', 'Week')
run_ljung_box_test(gru_metrics, 'comp', 14, 'GRU')
print(gru_metrics['comp'])

Plot saved to 'outputs/plots/gru_residuals_analysis_week'.
= Ljung-Box Test: GRU =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
                true           pred
2024-02-23   51094.0   52102.270102
2024-02-24   51429.0   51385.767436
2024-02-25   51566.0   51913.874995
2024-02-26   53644.0   52043.146106
2024-02-27   57165.0   51614.246608
...              ...            ...
2025-06-18  104922.0  108906.401057
2025-06-19  104688.0  105953.207550
2025-06-20  103316.0  106341.020881
2025-06-21  102139.0  105685.646415
2025-06-22  102958.0  105838.967964

[486 rows x 2 columns]


In [ ]:
xgb_params = model_hps['XGB']['weekly']

xgb_metrics, xgb_pred_out = train_and_forecast_with_best_xgb_model(train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                   xgb_params, FULL_COLS, TARGET_COL,
                                                                   INPUT_WINDOW, TARGET_WINDOW,
                                                                   FORECAST_HORIZON, 'Days')

= XGBoost Model Training, Evaluation, and Forecasting on Test Set (Horizon: 7 Days) =
--- Fitting XGBoost Model ---

--- Preparing Data for XGBoost ---


Training: 100%|██████████| 7/7 [07:24<00:00, 63.46s/it]



--- Evaluating XGBoost Model ---


Walk-Forward Validation: 100%|██████████| 486/486 [00:02<00:00, 193.17it/s]

- Mean Absolute Percentage Error (MAPE): 5.1270%
- Directional Accuracy (DA): 51.8519%

--- Generating Final XGBoost (Threshold: 0.5%) Forecast ---
- Forecast for 2025-06-29: $102673.95
- Directional Signal for 2025-06-29: 0.0


In [ ]:
plot_residuals_analysis(xgb_metrics, 'comp', 14, 'XGB', 'Week')
run_ljung_box_test(xgb_metrics, 'comp', 14, 'XGB')
print(xgb_metrics['comp'])

Plot saved to 'outputs/plots/xgb_residuals_analysis_week'.
= Ljung-Box Test: XGB =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
                true           pred
2024-02-23   51094.0   51900.718876
2024-02-24   51429.0   51259.202391
2024-02-25   51566.0   52107.378954
2024-02-26   53644.0   52193.319451
2024-02-27   57165.0   51657.281823
...              ...            ...
2025-06-18  104922.0  106841.500571
2025-06-19  104688.0  104699.876775
2025-06-20  103316.0  105636.869457
2025-06-21  102139.0  103568.786895
2025-06-22  102958.0  105562.000046

[486 rows x 2 columns]


In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(f'lstm_validation.weights.h5', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True)
lr_scheduler_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=7)

In [ ]:
callbacks_fit = [checkpoint_cb, early_stopping_cb, lr_scheduler_cb]
callbacks_eval_and_pred = [checkpoint_cb]

In [ ]:
xgb_model = XGBForecaster(train_prep_ds, valid_prep_ds,
                          FULL_COLS, TARGET_COL,
                          INPUT_WINDOW, TARGET_WINDOW)
xgb_params = {
    'params': model_hps['XGB']['weekly']
}

lstm_model = LSTMForecaster(train_prep_ds, valid_prep_ds,
                            FULL_COLS, TARGET_COL,
                            INPUT_WINDOW, TARGET_WINDOW)
lstm_model._prepare_data()
lstm_hps = kt.HyperParameters()
lstm_hps.values = model_hps['LSTM']['weekly']
lstm_hypermodel = RNNHyperModel(lstm_model.n_model_cols, TARGET_WINDOW, 'lstm').build(lstm_hps)
lstm_params_fit = {
    'model': lstm_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_fit
}
lstm_params_eval_and_pred = {
    'model': lstm_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_eval_and_pred
}

base_models = [lstm_model, xgb_model]

base_model_params_fit = {
    'forecaster_lstm': lstm_params_fit,
    'forecaster_xgb': xgb_params
}

base_model_params_eval_and_pred = {
    'forecaster_lstm': lstm_params_eval_and_pred,
    'forecaster_xgb': xgb_params
}

stacked_metrics_xl, stacked_pred_out_xl = train_and_forecast_with_stacked_ensemble_model(base_models, LinearRegression(),
                                                                                         test_prep_ds, base_model_params_fit, base_model_params_eval_and_pred,
                                                                                         FORECAST_HORIZON, 'Days')

--- Preparing Data for LSTM ---
= Stacked Ensemble Model Training, Evaluation, and Forecasting (Horizon: 7 Days) =
= Step 1: Fitting Stacked Ensemble Model =

--- Fitting LSTM Model ---
Epoch 1/100
121/121 - 24s - 199ms/step - loss: 0.3846 - mae: 0.7108 - val_loss: 0.1399 - val_mae: 0.3742 - learning_rate: 0.0051
Epoch 2/100
121/121 - 10s - 85ms/step - loss: 0.3216 - mae: 0.6172 - val_loss: 0.1377 - val_mae: 0.3686 - learning_rate: 0.0051
Epoch 3/100
121/121 - 9s - 76ms/step - loss: 0.3215 - mae: 0.6164 - val_loss: 0.1377 - val_mae: 0.3688 - learning_rate: 0.0051
Epoch 4/100
121/121 - 9s - 73ms/step - loss: 0.3214 - mae: 0.6165 - val_loss: 0.1365 - val_mae: 0.3659 - learning_rate: 0.0051
Epoch 5/100
121/121 - 9s - 75ms/step - loss: 0.3223 - mae: 0.6179 - val_loss: 0.1349 - val_mae: 0.3620 - learning_rate: 0.0051
Epoch 6/100
121/121 - 8s - 68ms/step - loss: 0.3217 - mae: 0.6161 - val_loss: 0.1346 - val_mae: 0.3616 - learning_rate: 0.0051
Epoch 7/100
121/121 - 9s - 75ms/step - loss: 0.31

Walk-Forward Validation: 100%|██████████| 485/485 [00:37<00:00, 12.77it/s]


- Mean Absolute Percentage Error (MAPE): 4.7065%
- Directional Accuracy (DA): 54.4330%
--- Fitting XGBoost Model ---

--- Preparing Data for XGBoost ---


Training: 100%|██████████| 7/7 [07:04<00:00, 60.64s/it]



--- Evaluating XGBoost Model ---


...

- Mean Absolute Percentage Error (MAPE): 5.0949%
- Directional Accuracy (DA): 49.3827%
--- Fitting XGBoost Model ---

--- Preparing Data for XGBoost ---


Training: 100%|██████████| 7/7 [07:27<00:00, 63.87s/it]



--- Evaluating XGBoost Model ---


Walk-Forward Validation: 100%|██████████| 486/486 [00:00<00:00, 550.25it/s]


- Mean Absolute Percentage Error (MAPE): 5.1270%
- Directional Accuracy (DA): 51.8519%

--- Evaluating Stacked Ensemble Model ---
- Mean Absolute Percentage Error (MAPE): 5.0958%
- Directional Accuracy (DA): 55.1440%
= Step 3: Generating Final Stacked Ensemble (Threshold: 0.5%) Forecast =

--- Generating Final LSTM (Threshold: 0.5%) Forecast ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
- Forecast for 2025-06-29: $102994.58
- Directional Signal for 2025-06-29: 0.0

--- Generating Final XGBoost (Threshold: 0.5%) Forecast ---
- Forecast for 2025-06-29: $102673.95
- Directional Signal for 2025-06-29: 0.0
- Forecast for 2025-06-29: $104202.00
- Directional Signal for 2025-06-29: 1.0


In [ ]:
plot_residuals_analysis(stacked_metrics_xl, 'comp', 14, 'Stacked Ensemble (LSTM+XGB)', 'Week')
run_ljung_box_test(stacked_metrics_xl, 'comp', 14, 'Stacked Ensemble (LSTM+XGB)')
print(stacked_metrics_xl['comp'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(f'gru_validation.weights.h5', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True)
lr_scheduler_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=7)

In [ ]:
callbacks_fit = [checkpoint_cb, early_stopping_cb, lr_scheduler_cb]
callbacks_eval_and_pred = [checkpoint_cb]

In [ ]:
xgb_model = XGBForecaster(train_prep_ds, valid_prep_ds,
                          FULL_COLS, TARGET_COL,
                          INPUT_WINDOW, TARGET_WINDOW)
xgb_params = {
    'params': model_hps['XGB']['weekly']
}

gru_model = GRUForecaster(train_prep_ds, valid_prep_ds,
                          FULL_COLS, TARGET_COL,
                          INPUT_WINDOW, TARGET_WINDOW)
gru_model._prepare_data()
gru_hps = kt.HyperParameters()
gru_hps.values = model_hps['GRU']['weekly']
gru_hypermodel = RNNHyperModel(gru_model.n_model_cols, TARGET_WINDOW, 'gru').build(gru_hps)
gru_params_fit = {
    'model': gru_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_fit
}
gru_params_eval_and_pred = {
    'model': gru_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_eval_and_pred
}

base_models = [gru_model, xgb_model]

base_model_params_fit = {
    'forecaster_gru': gru_params_fit,
    'forecaster_xgb': xgb_params
}

base_model_params_eval_and_pred = {
    'forecaster_gru': gru_params_eval_and_pred,
    'forecaster_xgb': xgb_params
}

stacked_metrics_xg, stacked_pred_out_xg = train_and_forecast_with_stacked_ensemble_model([gru_model, xgb_model], LinearRegression(),
                                                                                          test_prep_ds, base_model_params_fit, base_model_params_eval_and_pred,
                                                                                          FORECAST_HORIZON, 'Days')

--- Preparing Data for GRU ---
= Stacked Ensemble Model Training, Evaluation, and Forecasting (Horizon: 7 Days) =
= Step 1: Fitting Stacked Ensemble Model =

--- Fitting GRU Model ---
Epoch 1/100
121/121 - 19s - 156ms/step - loss: 1.1303 - mae: 0.7347 - val_loss: 0.1605 - val_mae: 0.3744 - learning_rate: 0.0051
Epoch 2/100
121/121 - 9s - 74ms/step - loss: 0.3317 - mae: 0.6180 - val_loss: 0.1415 - val_mae: 0.3670 - learning_rate: 0.0051
Epoch 3/100
121/121 - 10s - 84ms/step - loss: 0.3255 - mae: 0.6162 - val_loss: 0.1382 - val_mae: 0.3647 - learning_rate: 0.0051
Epoch 4/100
121/121 - 8s - 62ms/step - loss: 0.3237 - mae: 0.6154 - val_loss: 0.1379 - val_mae: 0.3644 - learning_rate: 0.0051
Epoch 5/100
121/121 - 10s - 84ms/step - loss: 0.3228 - mae: 0.6156 - val_loss: 0.1370 - val_mae: 0.3638 - learning_rate: 0.0051
Epoch 6/100
121/121 - 9s - 75ms/step - loss: 0.3222 - mae: 0.6153 - val_loss: 0.1383 - val_mae: 0.3679 - learning_rate: 0.0051
Epoch 7/100
121/121 - 9s - 75ms/step - loss: 0.322

Walk-Forward Validation: 100%|██████████| 485/485 [00:38<00:00, 12.72it/s]


- Mean Absolute Percentage Error (MAPE): 4.7061%
- Directional Accuracy (DA): 54.4330%
--- Fitting XGBoost Model ---

--- Preparing Data for XGBoost ---


Training: 100%|██████████| 7/7 [07:14<00:00, 62.13s/it]



--- Evaluating XGBoost Model ---


...

Training: 100%|██████████| 7/7 [07:28<00:00, 64.11s/it]



--- Evaluating XGBoost Model ---


Walk-Forward Validation: 100%|██████████| 486/486 [00:01<00:00, 304.25it/s]

- Mean Absolute Percentage Error (MAPE): 5.1270%
- Directional Accuracy (DA): 51.8519%

--- Evaluating Stacked Ensemble Model ---
- Mean Absolute Percentage Error (MAPE): 5.1061%
- Directional Accuracy (DA): 54.7325%
= Step 3: Generating Final Stacked Ensemble (Threshold: 0.5%) Forecast =

--- Generating Final GRU (Threshold: 0.5%) Forecast ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


- Forecast for 2025-06-29: $103186.71
- Directional Signal for 2025-06-29: 0.0

--- Generating Final XGBoost (Threshold: 0.5%) Forecast ---
- Forecast for 2025-06-29: $102673.95
- Directional Signal for 2025-06-29: 0.0
- Forecast for 2025-06-29: $104339.08
- Directional Signal for 2025-06-29: 1.0


In [ ]:
plot_residuals_analysis(stacked_metrics_xg, 'comp', 14, 'Stacked Ensemble (GRU+XGB)', 'Week')
run_ljung_box_test(stacked_metrics_xg, 'comp', 14, 'Stacked Ensemble (GRU+XGB)')
print(stacked_metrics_xg['comp'])

In [ ]:
true_values = naive_metrics['comp']['true']

naive_preds = naive_metrics['comp']['pred'].values
arima_preds = arima_metrics['comp']['pred'].values
lstm_preds = lstm_metrics['comp']['pred'].values
gru_preds = gru_metrics['comp']['pred'].values
xgb_preds = xgb_metrics['comp']['pred'].values
stacked_xl_preds = stacked_metrics_xl['comp']['pred'].values
stacked_xg_preds = stacked_metrics_xg['comp']['pred'].values

all_preds = {
    'Naive': naive_preds,
    'ARIMA': arima_preds,
    'LSTM': lstm_preds,
    'GRU': gru_preds,
    'XGBoost': xgb_preds,
    'StackedEnsembleXL': stacked_xl_preds,
    'StackedEnsembleXG': stacked_xg_preds
}

In [ ]:
run_all_dm_tests(true_values, all_preds, norm=1)

= Running All Model Pairwise Diebold-Mariano Tests =


--- Diebold-Mariano Test: Naive vs. StackedEnsembleXL ---
- Reshaping data from (486,) to (243, 2) for DM test compatibility.
- p-value: 0.7142
- Conclusion: The difference is not statistically significant (p >= 0.05).
- Result: Neither model can be considered superior to the other.


--- Diebold-Mariano Test: Naive vs. StackedEnsembleXG ---
- Reshaping data from (486,) to (243, 2) for DM test compatibility.
- p-value: 0.6694
- Conclusion: The difference is not statistically significant (p >= 0.05).
- Result: Neither model can be considered superior to the other.


--- Diebold-Mariano Test: StackedEnsembleXL vs. StackedEnsembleXG ---
- Reshaping data from (486,) to (243, 2) for DM test compatibility.
- p-value: 0.4462
- Conclusion: The difference is not statistically significant (p >= 0.05).
- Result: Neither model can be considered superior to the other.


= All pairwise Diebold-Mariano tests complete. =


In [123]:
all_model_metrics = {
    'Naive': naive_metrics,
    'ARIMA': arima_metrics,
    'LSTM': lstm_metrics,
    'GRU': gru_metrics,
    'XGB': xgb_metrics,
    'Stacked Ensemble (LSTM+XGB)': stacked_metrics_xl,
    'Stacked Ensemble (GRU+XGB)': stacked_metrics_xg
}

In [ ]:
display_metrics(all_model_metrics)

In [ ]:
display_metrics(all_model_metrics, 'mape')

### Model Comparisons (1-Month)

In [ ]:
tf.random.set_seed(42)

In [ ]:
FULL_COLS = list(train_prep_ds.columns)
TARGET_COL = 'close'
WINDOW_SIZE = 30
INPUT_WINDOW = 30
TARGET_WINDOW = 30
FORECAST_HORIZON = 30

In [ ]:
loaded_hps = load_config(CONFIG_FILE_PATH)

In [ ]:
naive_metrics, naive_pred_out = train_and_forecast_with_naive_model(test_prep_ds, TARGET_COL, FORECAST_HORIZON, 'Test', 'Days')

In [ ]:
plot_residuals_analysis(naive_metrics, 'comp', 90, 'NAIVE', 'Month')
run_ljung_box_test(naive_metrics, 'comp', 90, 'NAIVE')
print(naive_metrics['comp'])

In [ ]:
arima_order = tuple(model_hps['ARIMA']['monthly']['order'])
arima_metrics, arima_pred_out = train_and_forecast_with_best_arima_model(train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                         arima_order, TARGET_COL, WINDOW_SIZE,
                                                                         FORECAST_HORIZON, 'Days')

In [ ]:
plot_residuals_analysis(arima_metrics, 'comp', 90, 'ARIMA', 'Month')
run_ljung_box_test(arima_metrics, 'comp', 90, 'ARIMA')
print(arima_metrics['comp'])

In [ ]:
lstm_hps = kt.HyperParameters()
lstm_hps.values = model_hps['LSTM']['monthly']

lstm_metrics, lstm_pred_out = train_and_forecast_with_best_rnn_model('lstm', train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                    lstm_hps, FULL_COLS, TARGET_COL, FORECAST_HORIZON,
                                                                    INPUT_WINDOW, TARGET_WINDOW, 'Days')

In [ ]:
plot_residuals_analysis(lstm_metrics, 'comp', 90, 'LSTM', 'Month')
run_ljung_box_test(lstm_metrics, 'comp', 90, 'LSTM')
print(lstm_metrics['comp'])

In [ ]:
gru_hps = kt.HyperParameters()
gru_hps.values = model_hps['GRU']['monthly']

gru_metrics, gru_pred_out = train_and_forecast_with_best_rnn_model('gru', train_prep_ds, valid_prep_ds, test_prep_ds,
                                                                   gru_hps, FULL_COLS, TARGET_COL, FORECAST_HORIZON,
                                                                   INPUT_WINDOW, TARGET_WINDOW, 'Days')

In [ ]:
plot_residuals_analysis(gru_metrics, 'comp', 90, 'GRU', 'Month')
run_ljung_box_test(gru_metrics, 'comp', 90, 'GRU')
print(gru_metrics['comp'])

In [ ]:
xgb_params = loaded_hps['XGB']['monthly']

xgb_metrics, xgb_pred_out = train_and_forecast_with_best_xgb_model(train_prep_ds_daily, valid_prep_ds_daily, test_prep_ds_daily,
                                                                   xgb_params, FULL_COLS, TARGET_COL,
                                                                   INPUT_WINDOW, TARGET_WINDOW,
                                                                   FORECAST_HORIZON, 'Days')

In [ ]:
plot_residuals_analysis(xgb_metrics, 'comp', 90, 'XGB', 'Month')
run_ljung_box_test(xgb_metrics, 'comp', 90, 'XGB')
print(xgb_metrics['comp'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(f'lstm_validation.weights.h5', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True)
lr_scheduler_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=7)

In [ ]:
callbacks_fit = [checkpoint_cb, early_stopping_cb, lr_scheduler_cb]
callbacks_eval_and_pred = [checkpoint_cb]

In [ ]:
xgb_model = XGBForecaster(train_prep_ds, valid_prep_ds,
                          FULL_COLS, TARGET_COL,
                          INPUT_WINDOW, TARGET_WINDOW)
xgb_params = {
    'params': model_hps['XGB']['weekly']
}

lstm_model = LSTMForecaster(train_prep_ds, valid_prep_ds,
                            FULL_COLS, TARGET_COL,
                            INPUT_WINDOW, TARGET_WINDOW)
lstm_model._prepare_data()
lstm_hps = kt.HyperParameters()
lstm_hps.values = model_hps['LSTM']['weekly']
lstm_hypermodel = RNNHyperModel(lstm_model.n_model_cols, TARGET_WINDOW, 'lstm').build(lstm_hps)
lstm_params_fit = {
    'model': lstm_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_fit
}
lstm_params_eval_and_pred = {
    'model': lstm_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_eval_and_pred
}

base_models = [lstm_model, xgb_model]

base_model_params_fit = {
    'forecaster_lstm': lstm_params_fit,
    'forecaster_xgb': xgb_params
}

base_model_params_eval_and_pred = {
    'forecaster_lstm': lstm_params_eval_and_pred,
    'forecaster_xgb': xgb_params
}

stacked_metrics_xl, stacked_pred_out_xl = train_and_forecast_with_stacked_ensemble_model(base_models, LinearRegression(),
                                                                                         test_prep_ds, base_model_params_fit, base_model_params_eval_and_pred,
                                                                                         FORECAST_HORIZON, 'Days')

In [ ]:
plot_residuals_analysis(stacked_metrics_xl, 'comp', 90, 'Stacked Ensemble (LSTM+XGB)', 'Month')
run_ljung_box_test(stacked_metrics_xl, 'comp', 90, 'Stacked Ensemble (LSTM+XGB)')
print(stacked_metrics_xl['comp'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(f'gru_validation.weights.h5', save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=15, restore_best_weights=True)
lr_scheduler_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=7)

In [ ]:
callbacks_fit = [checkpoint_cb, early_stopping_cb, lr_scheduler_cb]
callbacks_eval_and_pred = [checkpoint_cb]

In [ ]:
xgb_model = XGBForecaster(train_prep_ds, valid_prep_ds,
                          FULL_COLS, TARGET_COL,
                          INPUT_WINDOW, TARGET_WINDOW)
xgb_params = {
    'params': model_hps['XGB']['monthly']
}

gru_model = LSTMForecaster(train_prep_ds, valid_prep_ds,
                            FULL_COLS, TARGET_COL,
                            INPUT_WINDOW, TARGET_WINDOW)
gru_model._prepare_data()
gru_hps = kt.HyperParameters()
gru_hps.values = model_hps['GRU']['monthly']
gru_hypermodel = RNNHyperModel(lstm_model.n_model_cols, TARGET_WINDOW, 'gru').build(lstm_hps)
gru_params_fit = {
    'model': gru_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_fit
}
gru_params_eval_and_pred = {
    'model': gru_hypermodel,
    'epochs': 100,
    'callbacks': callbacks_eval_and_pred
}

base_models = [gru_model, xgb_model]

base_model_params_fit = {
    'forecaster_gru': gru_params_fit,
    'forecaster_xgb': xgb_params
}

base_model_params_eval_and_pred = {
    'forecaster_gru': gru_params_eval_and_pred,
    'forecaster_xgb': xgb_params
}

stacked_metrics_xg, stacked_pred_out_xg = train_and_forecast_with_stacked_ensemble_model(base_models, LinearRegression(),
                                                                                         test_prep_ds, base_model_params_fit, base_model_params_eval_and_pred,
                                                                                         FORECAST_HORIZON, 'Days')

In [ ]:
plot_residuals_analysis(stacked_metrics_xg, 'comp', 90, 'Stacked Ensemble (GRU+XGB)', 'Month')
run_ljung_box_test(stacked_metrics_xg, 'comp', 90, 'Stacked Ensemble (GRU+XGB)')
print(stacked_metrics_xg['comp'])

In [ ]:
true_values = naive_metrics['comp']['true']

naive_preds = naive_metrics['comp']['pred'].values
arima_preds = arima_metrics['comp']['pred'].values
lstm_preds = lstm_metrics['comp']['pred'].values
gru_preds = gru_metrics['comp']['pred'].values
xgb_preds = xgb_metrics['comp']['pred'].values
stacked_xl_preds = stacked_metrics_xl['comp']['pred'].values
stacked_xg_preds = stacked_metrics_xg['comp']['pred'].values

all_preds = {
    'Naive': naive_preds,
    'ARIMA': arima_preds,
    'LSTM': lstm_preds,
    'GRU': gru_preds,
    'XGBoost': xgb_preds,
    'StackedEnsembleXL': stacked_xl_preds,
    'StackedEnsembleXG': stacked_xg_preds
}

In [ ]:
run_all_dm_tests(true_values, all_preds, norm=1)

= Running All Model Pairwise Diebold-Mariano Tests =


--- Diebold-Mariano Test: Naive vs. ARIMA ---
- Reshaping data from (462,) to (231, 2) for DM test compatibility.
- p-value: 0.9856
- Conclusion: The difference is not statistically significant (p >= 0.05).
- Result: Neither model can be considered superior to the other.


--- Diebold-Mariano Test: Naive vs. LSTM ---
- Reshaping data from (462,) to (231, 2) for DM test compatibility.
- p-value: 0.0328
Conclusion: The difference is statistically significant (p < 0.05).
- Result: LSTM is significantly better than Naive.


--- Diebold-Mariano Test: Naive vs. GRU ---
- Reshaping data from (462,) to (231, 2) for DM test compatibility.
- p-value: 0.0397
Conclusion: The difference is statistically significant (p < 0.05).
- Result: GRU is significantly better than Naive.


--- Diebold-Mariano Test: Naive vs. XGBoost ---
- Reshaping data from (462,) to (231, 2) for DM test compatibility.
- p-value: 0.9925
- Conclusion: The difference is not

In [ ]:
all_model_metrics = {
    'Naive': naive_metrics,
    'ARIMA': arima_metrics,
    'LSTM': lstm_metrics,
    'GRU': gru_metrics,
    'XGB': xgb_metrics,
    'Stacked Ensemble (LSTM+XGB)': stacked_metrics_xl,
    'Stacked Ensemble (GRU+XGB)': stacked_metrics_xg
}

In [ ]:
display_metrics(all_model_metrics)

--- Model Performance (Sorted by DA, higher is better) ---

Model                       | DA         | MAPE      
----------------------------|------------|------------
Stacked Ensemble (LSTM+XGB) | 57.6674  % | 9.8349   %
Stacked Ensemble (GRU+XGB)  | 54.2117  % | 9.8231   %
XGB                         | 52.0518  % | 9.8985   %
LSTM                        | 50.3240  % | 9.8389   %
GRU                         | 50.3240  % | 9.8285   %
ARIMA                       | 49.0281  % | 10.3055  %
Naive                       | 42.2633  % | 9.8408   %


In [ ]:
display_metrics(all_model_metrics, 'mape')

--- Model Performance (Sorted by MAPE, lower is better) ---

Model                       | DA         | MAPE      
----------------------------|------------|------------
Stacked Ensemble (GRU+XGB)  | 54.2117  % | 9.8231   %
GRU                         | 50.3240  % | 9.8285   %
Stacked Ensemble (LSTM+XGB) | 57.6674  % | 9.8349   %
LSTM                        | 50.3240  % | 9.8389   %
Naive                       | 42.2633  % | 9.8408   %
XGB                         | 52.0518  % | 9.8985   %
ARIMA                       | 49.0281  % | 10.3055  %
